In [1]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

#loading the dataset
dataset = dset.CIFAR10(root="./data", download=True,
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
nc=3

dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, num_workers=2)

#checking the availability of cuda devices
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# number of gpu's available
ngpu = 1
# input noise dimension
nz = 100
# number of generator filters
ngf = 64
#number of discriminator filters
ndf = 64

Random Seed:  1417
Files already downloaded and verified


In [3]:
import os
import imageio

if torch.cuda.is_available():
    use_gpu = True
leave_log = True
if leave_log:
    result_dir = 'DCGAN_generated_images'
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)
        
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [4]:

### DCGAN의 생성자
class Generator(nn.Module):
    
    # 네트워크 구조
    def __init__(self,ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8,4,1,0,bias=False),
            nn.BatchNorm2d(64*8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(64*8, 64*4, 4, 2, 1,bias=False),
            nn.BatchNorm2d(64*4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
      
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),            
            nn.Tanh())
        
    # (batch_size x 100) 크기의 랜덤 벡터를 받아 
    # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
        if inputs.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(inputs)
            return output

In [5]:
netG = Generator(ngpu).to(device)
netG.apply(weights_init)
#load weights to test the model
#netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))
print(netG)


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [6]:
### DCGAN의 구분자
class Discriminator(nn.Module):
    
    # 네트워크 구조
    def __init__(self,ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
           nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid())
        
    # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
    # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
    def forward(self, inputs):
        if inputs.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(inputs)

        return output.view(-1, 1).squeeze(1)

In [7]:
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
#load weights to test the model 
#netD.load_state_dict(torch.load('weights/netD_epoch_24.pth'))
print(netD)


Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [8]:
criterion = nn.BCELoss()

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(128, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

In [9]:
# 학습 결과 시각화하기
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

    if type(data) == list:
	    data = np.concatenate(data)
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data, cmap='gray')

In [10]:
if leave_log:
    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    generated_images = []
    
z_fixed = Variable(torch.randn(5 * 5, 100), volatile=True)
if use_gpu:
    z_fixed = z_fixed.cuda()

C:\Users\wlska\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


In [11]:
niter = 100
g_loss = []
d_loss = []

for epoch in range(niter):
    for i, data in enumerate(dataloader ,0):
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)
        
        output = netD(real_cpu)
        errD_real = criterion(output,label)
        errD_real.backward()
        D_x = output.mean().item()
        
        noise = torch.randn(batch_size,nz,1,1,device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output=netD(fake.detach())
        errD_fake=criterion(output,label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' % (epoch, niter, i, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            print('saving the output')
            vutils.save_image(real_cpu,'output/real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            image_path='output/fake_samples_epoch_%03d.png' % (epoch)
            vutils.save_image(fake.detach(),image_path,normalize=True)
            generated_images.append(image_path)
       
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), 'weights/netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), 'weights/netD_epoch_%d.pth' % (epoch))
    
generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/DCGAN_generation.gif', generated_image_array, fps=10)

[0/100][0/391] Loss_D: 2.3301 Loss_G: 6.0773 D(x): 0.6807 D(G(z)): 0.7892 / 0.0045
saving the output
[0/100][1/391] Loss_D: 1.4274 Loss_G: 4.7702 D(x): 0.4906 D(G(z)): 0.3577 / 0.0137
[0/100][2/391] Loss_D: 1.3573 Loss_G: 5.2251 D(x): 0.6679 D(G(z)): 0.5030 / 0.0088
[0/100][3/391] Loss_D: 1.3064 Loss_G: 6.0889 D(x): 0.6872 D(G(z)): 0.4511 / 0.0040
[0/100][4/391] Loss_D: 1.1976 Loss_G: 6.5574 D(x): 0.6746 D(G(z)): 0.3860 / 0.0028
[0/100][5/391] Loss_D: 1.0784 Loss_G: 6.8501 D(x): 0.6620 D(G(z)): 0.3524 / 0.0019
[0/100][6/391] Loss_D: 1.1159 Loss_G: 7.8097 D(x): 0.7168 D(G(z)): 0.3949 / 0.0007
[0/100][7/391] Loss_D: 0.7269 Loss_G: 7.2352 D(x): 0.7262 D(G(z)): 0.2237 / 0.0012
[0/100][8/391] Loss_D: 0.8967 Loss_G: 8.8672 D(x): 0.7941 D(G(z)): 0.3843 / 0.0002
[0/100][9/391] Loss_D: 0.6277 Loss_G: 7.8727 D(x): 0.7510 D(G(z)): 0.1559 / 0.0007
[0/100][10/391] Loss_D: 0.8114 Loss_G: 9.5756 D(x): 0.7965 D(G(z)): 0.3501 / 0.0001
[0/100][11/391] Loss_D: 0.5012 Loss_G: 8.1709 D(x): 0.7663 D(G(z)): 

[0/100][98/391] Loss_D: 0.3974 Loss_G: 5.9043 D(x): 0.9159 D(G(z)): 0.2289 / 0.0059
[0/100][99/391] Loss_D: 0.2571 Loss_G: 6.0119 D(x): 0.9146 D(G(z)): 0.1214 / 0.0049
[0/100][100/391] Loss_D: 0.3759 Loss_G: 3.7794 D(x): 0.8140 D(G(z)): 0.0694 / 0.0317
saving the output
[0/100][101/391] Loss_D: 0.6766 Loss_G: 8.8563 D(x): 0.9115 D(G(z)): 0.3822 / 0.0012
[0/100][102/391] Loss_D: 1.3645 Loss_G: 3.1248 D(x): 0.4890 D(G(z)): 0.0151 / 0.0651
[0/100][103/391] Loss_D: 0.7640 Loss_G: 9.1118 D(x): 0.9372 D(G(z)): 0.4585 / 0.0004
[0/100][104/391] Loss_D: 1.1327 Loss_G: 5.1757 D(x): 0.4751 D(G(z)): 0.0075 / 0.0097
[0/100][105/391] Loss_D: 0.3074 Loss_G: 4.5781 D(x): 0.9256 D(G(z)): 0.1883 / 0.0133
[0/100][106/391] Loss_D: 0.5375 Loss_G: 6.3380 D(x): 0.8569 D(G(z)): 0.2471 / 0.0045
[0/100][107/391] Loss_D: 0.2740 Loss_G: 5.4523 D(x): 0.8546 D(G(z)): 0.0738 / 0.0101
[0/100][108/391] Loss_D: 0.3080 Loss_G: 5.7728 D(x): 0.9093 D(G(z)): 0.1644 / 0.0055
[0/100][109/391] Loss_D: 0.4368 Loss_G: 4.3414 D(

[0/100][197/391] Loss_D: 0.0997 Loss_G: 10.5985 D(x): 0.9190 D(G(z)): 0.0001 / 0.0001
[0/100][198/391] Loss_D: 0.0668 Loss_G: 6.5200 D(x): 0.9475 D(G(z)): 0.0021 / 0.0035
[0/100][199/391] Loss_D: 0.0845 Loss_G: 4.8651 D(x): 0.9838 D(G(z)): 0.0601 / 0.0106
[0/100][200/391] Loss_D: 0.2045 Loss_G: 8.5093 D(x): 0.9725 D(G(z)): 0.1496 / 0.0005
saving the output
[0/100][201/391] Loss_D: 0.2295 Loss_G: 6.3126 D(x): 0.8658 D(G(z)): 0.0062 / 0.0030
[0/100][202/391] Loss_D: 0.1080 Loss_G: 4.7820 D(x): 0.9527 D(G(z)): 0.0507 / 0.0116
[0/100][203/391] Loss_D: 0.2686 Loss_G: 9.1614 D(x): 0.9709 D(G(z)): 0.1811 / 0.0005
[0/100][204/391] Loss_D: 0.2766 Loss_G: 6.6461 D(x): 0.8318 D(G(z)): 0.0067 / 0.0083
[0/100][205/391] Loss_D: 0.2342 Loss_G: 4.9999 D(x): 0.9166 D(G(z)): 0.0700 / 0.0126
[0/100][206/391] Loss_D: 0.3490 Loss_G: 9.8682 D(x): 0.9651 D(G(z)): 0.2286 / 0.0001
[0/100][207/391] Loss_D: 1.0889 Loss_G: 2.4320 D(x): 0.4932 D(G(z)): 0.0032 / 0.2275
[0/100][208/391] Loss_D: 1.7603 Loss_G: 13.718

[0/100][294/391] Loss_D: 0.7108 Loss_G: 8.2983 D(x): 0.9845 D(G(z)): 0.4535 / 0.0004
[0/100][295/391] Loss_D: 1.1033 Loss_G: 3.7455 D(x): 0.5072 D(G(z)): 0.0024 / 0.0532
[0/100][296/391] Loss_D: 0.3460 Loss_G: 4.9029 D(x): 0.9779 D(G(z)): 0.2373 / 0.0139
[0/100][297/391] Loss_D: 0.2483 Loss_G: 5.6580 D(x): 0.9279 D(G(z)): 0.1399 / 0.0056
[0/100][298/391] Loss_D: 0.3943 Loss_G: 2.5550 D(x): 0.7560 D(G(z)): 0.0568 / 0.1178
[0/100][299/391] Loss_D: 1.2004 Loss_G: 11.9422 D(x): 0.9659 D(G(z)): 0.5943 / 0.0000
[0/100][300/391] Loss_D: 3.5985 Loss_G: 4.8113 D(x): 0.0919 D(G(z)): 0.0002 / 0.0357
saving the output
[0/100][301/391] Loss_D: 0.3441 Loss_G: 4.2073 D(x): 0.9627 D(G(z)): 0.2161 / 0.0427
[0/100][302/391] Loss_D: 0.3946 Loss_G: 6.4422 D(x): 0.9639 D(G(z)): 0.2559 / 0.0039
[0/100][303/391] Loss_D: 0.5802 Loss_G: 3.3590 D(x): 0.6730 D(G(z)): 0.0424 / 0.0735
[0/100][304/391] Loss_D: 0.4867 Loss_G: 4.0687 D(x): 0.9268 D(G(z)): 0.2924 / 0.0280
[0/100][305/391] Loss_D: 0.3077 Loss_G: 4.1183

[1/100][0/391] Loss_D: 0.2455 Loss_G: 5.0122 D(x): 0.8684 D(G(z)): 0.0786 / 0.0107
saving the output
[1/100][1/391] Loss_D: 0.4204 Loss_G: 3.1273 D(x): 0.7775 D(G(z)): 0.1191 / 0.0735
[1/100][2/391] Loss_D: 0.4952 Loss_G: 6.7715 D(x): 0.9068 D(G(z)): 0.2981 / 0.0016
[1/100][3/391] Loss_D: 0.3541 Loss_G: 4.5103 D(x): 0.7406 D(G(z)): 0.0096 / 0.0198
[1/100][4/391] Loss_D: 0.2408 Loss_G: 4.5499 D(x): 0.9379 D(G(z)): 0.1322 / 0.0199
[1/100][5/391] Loss_D: 0.2393 Loss_G: 5.3823 D(x): 0.9168 D(G(z)): 0.1236 / 0.0075
[1/100][6/391] Loss_D: 0.2903 Loss_G: 3.6720 D(x): 0.8313 D(G(z)): 0.0580 / 0.0443
[1/100][7/391] Loss_D: 0.3588 Loss_G: 6.4804 D(x): 0.9350 D(G(z)): 0.2224 / 0.0027
[1/100][8/391] Loss_D: 0.2328 Loss_G: 4.8868 D(x): 0.8370 D(G(z)): 0.0176 / 0.0181
[1/100][9/391] Loss_D: 0.1800 Loss_G: 3.9332 D(x): 0.9214 D(G(z)): 0.0779 / 0.0267
[1/100][10/391] Loss_D: 0.3607 Loss_G: 5.9065 D(x): 0.9262 D(G(z)): 0.2295 / 0.0040
[1/100][11/391] Loss_D: 0.2865 Loss_G: 4.2442 D(x): 0.8131 D(G(z)): 

...

[98/100][346/391] Loss_D: 0.0781 Loss_G: 7.4843 D(x): 0.9316 D(G(z)): 0.0007 / 0.0015
[98/100][347/391] Loss_D: 0.0430 Loss_G: 5.6315 D(x): 0.9981 D(G(z)): 0.0387 / 0.0068
[98/100][348/391] Loss_D: 0.0124 Loss_G: 8.0989 D(x): 0.9886 D(G(z)): 0.0007 / 0.0007
[98/100][349/391] Loss_D: 0.0129 Loss_G: 7.3879 D(x): 0.9886 D(G(z)): 0.0012 / 0.0015
[98/100][350/391] Loss_D: 0.0100 Loss_G: 6.0903 D(x): 0.9945 D(G(z)): 0.0044 / 0.0046
[98/100][351/391] Loss_D: 0.0168 Loss_G: 5.5252 D(x): 0.9941 D(G(z)): 0.0101 / 0.0079
[98/100][352/391] Loss_D: 0.0735 Loss_G: 7.3599 D(x): 0.9991 D(G(z)): 0.0663 / 0.0011
[98/100][353/391] Loss_D: 0.0359 Loss_G: 6.7121 D(x): 0.9833 D(G(z)): 0.0181 / 0.0025
[98/100][354/391] Loss_D: 0.3034 Loss_G: 5.6723 D(x): 0.7759 D(G(z)): 0.0013 / 0.0072
[98/100][355/391] Loss_D: 0.3260 Loss_G: 0.0436 D(x): 0.7670 D(G(z)): 0.0093 / 0.9605
[98/100][356/391] Loss_D: 3.5347 Loss_G: 15.9383 D(x): 1.0000 D(G(z)): 0.9234 / 0.0000
[98/100][357/391] Loss_D: 7.1440 Loss_G: 4.5676 D(x):

[99/100][52/391] Loss_D: 0.0962 Loss_G: 4.1705 D(x): 0.9659 D(G(z)): 0.0526 / 0.0300
[99/100][53/391] Loss_D: 0.0891 Loss_G: 4.4693 D(x): 0.9775 D(G(z)): 0.0593 / 0.0223
[99/100][54/391] Loss_D: 0.1524 Loss_G: 3.5571 D(x): 0.9017 D(G(z)): 0.0372 / 0.0528
[99/100][55/391] Loss_D: 0.1614 Loss_G: 3.2270 D(x): 0.9288 D(G(z)): 0.0760 / 0.0655
[99/100][56/391] Loss_D: 0.1124 Loss_G: 4.2138 D(x): 0.9750 D(G(z)): 0.0775 / 0.0262
[99/100][57/391] Loss_D: 0.1149 Loss_G: 4.4239 D(x): 0.9476 D(G(z)): 0.0500 / 0.0253
[99/100][58/391] Loss_D: 0.1413 Loss_G: 3.8092 D(x): 0.9106 D(G(z)): 0.0308 / 0.0459
[99/100][59/391] Loss_D: 0.1089 Loss_G: 3.8681 D(x): 0.9673 D(G(z)): 0.0624 / 0.0413
[99/100][60/391] Loss_D: 0.0991 Loss_G: 4.7782 D(x): 0.9892 D(G(z)): 0.0784 / 0.0161
[99/100][61/391] Loss_D: 0.1692 Loss_G: 3.9628 D(x): 0.8839 D(G(z)): 0.0214 / 0.0411
[99/100][62/391] Loss_D: 0.1067 Loss_G: 3.4556 D(x): 0.9458 D(G(z)): 0.0405 / 0.0581
[99/100][63/391] Loss_D: 0.1460 Loss_G: 4.7510 D(x): 0.9903 D(G(z

[99/100][149/391] Loss_D: 0.0465 Loss_G: 5.0720 D(x): 0.9820 D(G(z)): 0.0238 / 0.0141
[99/100][150/391] Loss_D: 0.0288 Loss_G: 5.2607 D(x): 0.9845 D(G(z)): 0.0125 / 0.0096
[99/100][151/391] Loss_D: 0.0274 Loss_G: 6.4492 D(x): 0.9755 D(G(z)): 0.0020 / 0.0033
[99/100][152/391] Loss_D: 0.0359 Loss_G: 4.6950 D(x): 0.9941 D(G(z)): 0.0290 / 0.0155
[99/100][153/391] Loss_D: 0.0368 Loss_G: 5.2023 D(x): 0.9796 D(G(z)): 0.0154 / 0.0105
[99/100][154/391] Loss_D: 0.0216 Loss_G: 5.6581 D(x): 0.9845 D(G(z)): 0.0056 / 0.0063
[99/100][155/391] Loss_D: 0.0286 Loss_G: 5.1034 D(x): 0.9822 D(G(z)): 0.0102 / 0.0114
[99/100][156/391] Loss_D: 0.0353 Loss_G: 5.2867 D(x): 0.9969 D(G(z)): 0.0303 / 0.0104
[99/100][157/391] Loss_D: 0.0220 Loss_G: 5.8046 D(x): 0.9861 D(G(z)): 0.0077 / 0.0058
[99/100][158/391] Loss_D: 0.0295 Loss_G: 5.3186 D(x): 0.9888 D(G(z)): 0.0175 / 0.0108
[99/100][159/391] Loss_D: 0.0151 Loss_G: 6.2753 D(x): 0.9889 D(G(z)): 0.0038 / 0.0039
[99/100][160/391] Loss_D: 0.0311 Loss_G: 5.0718 D(x): 

[99/100][246/391] Loss_D: 0.2390 Loss_G: 4.7616 D(x): 0.9566 D(G(z)): 0.1352 / 0.0229
[99/100][247/391] Loss_D: 0.4665 Loss_G: 2.1504 D(x): 0.7124 D(G(z)): 0.0479 / 0.2154
[99/100][248/391] Loss_D: 0.7657 Loss_G: 6.4204 D(x): 0.9716 D(G(z)): 0.4214 / 0.0040
[99/100][249/391] Loss_D: 0.5800 Loss_G: 3.5953 D(x): 0.6457 D(G(z)): 0.0120 / 0.0811
[99/100][250/391] Loss_D: 0.2653 Loss_G: 2.6648 D(x): 0.8879 D(G(z)): 0.0986 / 0.1403
[99/100][251/391] Loss_D: 0.3875 Loss_G: 5.0849 D(x): 0.9614 D(G(z)): 0.2323 / 0.0161
[99/100][252/391] Loss_D: 0.2847 Loss_G: 4.0278 D(x): 0.8338 D(G(z)): 0.0511 / 0.0476
[99/100][253/391] Loss_D: 0.2933 Loss_G: 3.1029 D(x): 0.8764 D(G(z)): 0.1067 / 0.0889
[99/100][254/391] Loss_D: 0.2223 Loss_G: 4.1998 D(x): 0.9473 D(G(z)): 0.1296 / 0.0315
[99/100][255/391] Loss_D: 0.2380 Loss_G: 4.2800 D(x): 0.8956 D(G(z)): 0.0973 / 0.0329
[99/100][256/391] Loss_D: 0.4672 Loss_G: 2.4076 D(x): 0.7678 D(G(z)): 0.0935 / 0.1744
[99/100][257/391] Loss_D: 0.3679 Loss_G: 5.3476 D(x): 

[99/100][343/391] Loss_D: 0.0610 Loss_G: 6.0015 D(x): 0.9911 D(G(z)): 0.0480 / 0.0050
[99/100][344/391] Loss_D: 0.1534 Loss_G: 2.6880 D(x): 0.8834 D(G(z)): 0.0132 / 0.1300
[99/100][345/391] Loss_D: 0.2013 Loss_G: 8.5363 D(x): 0.9957 D(G(z)): 0.1484 / 0.0005
[99/100][346/391] Loss_D: 0.3628 Loss_G: 2.7894 D(x): 0.7613 D(G(z)): 0.0039 / 0.1282
[99/100][347/391] Loss_D: 0.1511 Loss_G: 4.3151 D(x): 0.9824 D(G(z)): 0.1063 / 0.0250
[99/100][348/391] Loss_D: 0.2180 Loss_G: 3.6875 D(x): 0.9043 D(G(z)): 0.0889 / 0.0476
[99/100][349/391] Loss_D: 0.1526 Loss_G: 5.2468 D(x): 0.9752 D(G(z)): 0.1026 / 0.0115
[99/100][350/391] Loss_D: 0.0595 Loss_G: 5.8493 D(x): 0.9895 D(G(z)): 0.0415 / 0.0063
[99/100][351/391] Loss_D: 0.0561 Loss_G: 5.5466 D(x): 0.9536 D(G(z)): 0.0065 / 0.0086
[99/100][352/391] Loss_D: 0.0288 Loss_G: 5.1100 D(x): 0.9944 D(G(z)): 0.0223 / 0.0160
[99/100][353/391] Loss_D: 0.0596 Loss_G: 4.5302 D(x): 0.9647 D(G(z)): 0.0203 / 0.0213
[99/100][354/391] Loss_D: 0.0502 Loss_G: 4.9677 D(x): 